In [ ]:
# pip install pymysql

In [ ]:
# pip install mariadb

In [1]:
import pandas as pd
import copy

In [2]:
from mariadb import MariaDB
with open('./aws_config','r',encoding='utf-8') as f:
    db_config = {lines.split('=')[0].strip():lines.split('=')[1].strip() for lines in f.readlines()}
db = MariaDB(db_config)

In [ ]:
# rating_dart = pd.read_excel('rating_dart_total.xlsx')
# dart = rating_dart[['회사명', '년도', '총점수']].copy()
# dart.rename(columns={'총점수':'다트점수'}, inplace=True)
# dart.set_index(['회사명', '년도'], inplace=True)
# dart

In [3]:
rating_patent = pd.read_csv('특허_점수.csv', sep='\t')
patent = rating_patent[['회사명', '년도', '점수']].copy()
patent.rename(columns={'점수':'특허점수'}, inplace=True)
patent.set_index(['회사명', '년도'], inplace=True)
patent

특허점수
회사명     년도        
CJ ENM  2016   1.0
        2020   1.0
        2022   1.0
CJ프레시웨이 2014   1.0
        2015   1.0
...            ...
휴온스글로벌  2012   1.0
        2014   1.0
        2015   1.0
        2021   1.0
        2023   1.0

[2605 rows x 1 columns]

In [4]:
rating_green = pd.read_csv('녹색점수.csv')
green = rating_green[['기업명', '인증연도', '점수']].copy()
green.columns = ['회사명', '년도', '녹색점수']
green.set_index(['회사명', '년도'], inplace=True)
green

녹색점수
회사명   년도        
국일제지  2021     2
뉴보텍   2024     2
대원산업  2024     2
대창스틸  2023     2
      2024     2
...          ...
하츠    2024     2
한라IMS 2024     2
한탑    2024     2
해성산업  2021     2
효성오앤비 2022     2

[63 rows x 1 columns]

In [5]:
rating_recall = pd.read_csv('리콜점수.csv')
recall = rating_recall[['기업명', '인증연도', '점수']].copy()
recall.columns = ['회사명', '년도', '리콜점수']
recall.set_index(['회사명', '년도'], inplace=True)
recall

리콜점수
회사명     년도        
더네이쳐홀딩스 2019     4
동일금속    2021     4
로보로보    2015     4
머큐리     2014     3
바른손     2015     4
배럴      2019     4
비엠티     2018     4
삼천리자전거  2019     4
        2022     2
        2023     4
        2023     4
서울리거    2016     4
스튜디오삼익  2021     4
        2023     4
알톤스포츠   2022     4
        2023     4
        2023     4
에코아이    2015     4
        2017     4
        2023     4
오로라     2021     4
정다운     2016     2
        2017     3
제로투세븐   2015     4
제이에스티나  2019     4
토박스코리아  2017     4
        2019     3
하츠      2024     4

In [6]:
rating_bert = pd.read_csv('t_kosdaq_exec_df.csv')
bert = rating_bert[['기업', '년도', 'E', 'S', 'G', 'P', 'N']].copy()
bert['E*P'] = bert['E'] * bert['P'] * 8
bert['S*P'] = bert['S'] * bert['P'] * 20
bert['G*P'] = bert['G'] * bert['P'] * 15

# # '년도'가 2022 또는 2023인 경우 다른 계산식 적용
# import numpy as np
# bert['G*P'] = np.where(
#     (bert['년도'] == 2022) | (bert['년도'] == 2023),
#     bert['G'] * bert['P'] * 20,  # 다트 있는 경우 (2023, 2022)
#     bert['G'] * bert['P'] * 25   # 다트 없는 경우
# )

bert = bert[['기업', '년도', 'E*P', 'S*P', 'G*P', 'P', 'N']].copy()
bert.columns = ['회사명', '년도', 'bert_e', 'bert_s', 'bert_g', 'Company_Pos', 'Company_Neg']
bert.set_index(['회사명', '년도'], inplace=True)
bert

bert_e    bert_s    bert_g  Company_Pos  Company_Neg
회사명        년도                                                          
오상헬스케어     2019  1.967508  5.702600  4.391986       0.5289       0.4711
           2020  2.117860  6.087304  4.774793       0.5719       0.4281
           2021  2.267218  6.390324  5.120523       0.6121       0.3879
           2022  2.184249  6.231950  4.919867       0.5897       0.4103
           2023  2.008548  5.734852  4.463859       0.5397       0.4603
...                   ...       ...       ...          ...          ...
하나32호스팩    2020  2.774912  7.609563  6.644869       0.7605       0.2395
           2019  2.502975  7.128604  5.722144       0.6806       0.3194
IBKS제24호스팩 2024  2.457009  6.781482  5.647028       0.6642       0.3358
BF랩스       2024  2.408966  6.769836  5.493155       0.6522       0.3478
SG&G       2024  2.235481  6.560494  5.123684       0.6051       0.3949

[9824 rows x 5 columns]

In [7]:
dff = pd.concat([green, recall, bert, patent],axis=0)
# dff = pd.concat([dart, green, recall, bert],axis=0)

# 회사명을 기준으로 정렬
dff.sort_values(by=['회사명', '년도'], inplace=True)
dff = dff.groupby(['회사명', '년도']).sum()
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수  
회사명 년도          
3S  2019   0.0  
    2020   0.0  
    2021   0.0  
    2022   0.0  
    2023   0.0  
...        ...  
힘스  2020   0.0  
    2021   0.0  
    2022   0.0  
    2023   0.0  
    2024   0.0  

[11379 rows x 8 columns]

In [8]:
rating_job = pd.read_csv('kosdaq_jobplanet_numer_data_result.csv')
job = rating_job[['회사명', 's_point', 'g_point']].copy()
job.columns = ['회사명', 'job_s', 'job_g']
job

,회사명,job_s,job_g
0,아이엠비디엑스,1.703667,2.130000
1,엔젤로보틱스,1.778000,1.742000
2,제이투케이바이오,4.633333,8.666667
3,삼현,2.138500,1.831333
4,오상헬스케어,1.200000,1.157333
...,...,...,...
1714,한국전자인증,3.963750,1.973333
1715,BGF에코머티리얼즈,0.000000,0.000000
1716,NPX,0.000000,0.000000
1717,하나32호스팩,0.000000,0.000000


In [9]:
# job 데이터를 MultiIndex 데이터프레임에 채워 넣기
for idx, row in job.iterrows():
    company = row['회사명']
    if company in dff.index.get_level_values(0):
        for year in dff.loc[company].index:
            dff.loc[(company, year), 'job_s'] = row['job_s']
            dff.loc[(company, year), 'job_g'] = row['job_g']
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s  job_g  
회사명 년도                           
3S  2019   0.0  0.986000  3.538  
    2020   0.0  0.986000  3.538  
    2021   0.0  0.986000  3.538  
    2022   0.0  0.986000  3.538  
    2023   0.0  0.986000  3.538  
...        ...       ...    ...  
힘스  2020   0.0  1.085083  2.592  
    2021   0.0  1.085083  2.592  
    2022   0.0  1.085083  2.592  
    2023   0.0  1.085083  2.592  
    2024   0.0  1.085083  2.592  

[11379 rows x 10 columns]

In [111]:
# # 결측치인 행의 인덱스 반환
# dff['job_s'][dff['job_s'].isnull()]

In [112]:
# # 결측치는 0으로 채우기
# dff['job_s'] = dff['job_s'].fillna(0)
# dff['job_g'] = dff['job_g'].fillna(0)
# dff

In [10]:
# '년도' 인덱스를 검사하여 2023, 2022가 아닌 경우 'job_g' 값을 1.5배로 조정
# dff['job_g'] = dff.apply(lambda x: x['job_g'] * 1.5 if x.name[1] not in [2023, 2022] else x['job_g'], axis=1)
dff['job_g'] *= 3
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s   job_g  
회사명 년도                            
3S  2019   0.0  0.986000  10.614  
    2020   0.0  0.986000  10.614  
    2021   0.0  0.986000  10.614  
    2022   0.0  0.986000  10.614  
    2023   0.0  0.986000  10.614  
...        ...       ...     ...  
힘스  2020   0.0  1.085083   7.776  
    2021   0.0  1.085083   7.776  
    2022   0.0  1.085083   7.776  
    2023   0.0  1.085083   7.776  
    2024   0.0  1.085083   7.776  

[11379 rows x 10 columns]

In [11]:
# 점수 합계 구하기
dff['E점수(100)'] = ((dff['bert_e'] + dff['녹색점수'] + dff['특허점수'])/15)*100
dff['S점수(100)'] = ((dff['bert_s'] + dff['job_s'] + dff['리콜점수'])/40)*100
# dff['G점수(100)'] = ((dff['bert_g'] + dff['job_g'] + dff['다트점수'])/45)*100
dff['G점수(100)'] = ((dff['bert_g'] + dff['job_g'])/45)*100
# dff['총점수(100)'] = dff['다트점수']  + dff['녹색점수'] + dff['리콜점수'] + dff['bert_e'] + dff['bert_s'] + dff['bert_g'] + dff['job_s'] + dff['job_g'] + dff['특허점수']
dff['총점수(100)'] = dff['녹색점수'] + dff['리콜점수'] + dff['bert_e'] + dff['bert_s'] + dff['bert_g'] + dff['job_s'] + dff['job_g'] + dff['특허점수']
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s   job_g   E점수(100)   S점수(100)   G점수(100)   총점수(100)  
회사명 년도                                                                        
3S  2019   0.0  0.986000  10.614  14.468826  17.943107  34.441013  24.846023  
    2020   0.0  0.986000  10.614  14.589293  18.091751  34.543185  24.969527  
    2021   0.0  0.986000  10.614  14.758912  18.150320  34.631995  25.058362  
    2022   0.0  0.986000  10.614  15.201003  18.625480  35.015253  25.487206  
    2023   0.0  0.986000  10.614  15.743188  19.077887  35.425005  25.933885  
...        ...       ...     ...        ...        ...        ...        ...  
힘스  2020   0.0  1.085083   7.776  12.760202  16.481524  26.802744  20.567875  
    2021   0.0  1.085083   7.776  14.142528  17.889888  27.841597  21.806053  
    2022   0.0  1.085083   7.776  14.613312  18.225401  28.228980  22.185198  
    2023   0.0  1.085083   7.776  13.559127  17.132744  27.291791  21.168273  
    2024   0.0  1.085083   7.776  14.398129  18.078507  28.054123  22.015478  

[11379 rows x 14 columns]

In [12]:
# 회사명과 년도 기준으로 데이터프레임 정렬
dff.sort_values(by=['회사명', '년도'], inplace=True)

# 각 점수에 대한 이전 년도와의 차이 계산
for column in ['E점수(100)', 'S점수(100)', 'G점수(100)', '총점수(100)']:
    dff[f'{column} 차이'] = dff.groupby('회사명')[column].diff()
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s   job_g   E점수(100)   S점수(100)   G점수(100)   총점수(100)  \
회사명 년도                                                                         
3S  2019   0.0  0.986000  10.614  14.468826  17.943107  34.441013  24.846023   
    2020   0.0  0.986000  10.614  14.589293  18.091751  34.543185  24.969527   
    2021   0.0  0.986000  10.614  14.758912  18.150320  34.631995  25.058362   
    2022   0.0  0.986000  10.614  15.201003  18.625480  35.015253  25.487206   
    2023   0.0  0.986000  10.614  15.743188  19.077887  35.425005  25.933885   
...        ...       ...     ...        ...        ...        ...        ...   
힘스  2020   0.0  1.085083   7.776  12.760202  16.481524  26.802744  20.567875   
    2021   0.0  1.085083   7.776  14.142528  17.889888  27.841597  21.806053   
    2022   0.0  1.085083   7.776  14.613312  18.225401  28.228980  22.185198   
    2023   0.0  1.085083   7.776  13.559127  17.132744  27.291791  21.168273   
    2024   0.0  1.085083   7.776  14.398129  18.078507  28.054123  22.015478   

          E점수(100) 차이  S점수(100) 차이  G점수(100) 차이  총점수(100) 차이  
회사명 년도                                                        
3S  2019          NaN          NaN          NaN          NaN  
    2020     0.120467     0.148644     0.102172     0.123505  
    2021     0.169619     0.058569     0.088810     0.088835  
    2022     0.442091     0.475160     0.383259     0.428844  
    2023     0.542186     0.452407     0.409752     0.446679  
...               ...          ...          ...          ...  
힘스  2020     0.715609     1.064022     0.694795     0.845608  
    2021     1.382326     1.408364     1.038853     1.238178  
    2022     0.470784     0.335513     0.387383     0.379145  
    2023    -1.054185    -1.092657    -0.937189    -1.016925  
    2024     0.839001     0.945763     0.762332     0.847205  

[11379 rows x 18 columns]

In [13]:
dff = dff.reset_index()
dff

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,0.0,0.0,2.170324,6.191243,4.884456,0.5843,0.4157,0.0,0.986000,10.614,14.468826,17.943107,34.441013,24.846023,NaN,NaN,NaN,NaN
1,3S,2020,0.0,0.0,2.188394,6.250700,4.930433,0.5898,0.4102,0.0,0.986000,10.614,14.589293,18.091751,34.543185,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,0.0,0.0,2.213837,6.274128,4.970398,0.5964,0.4036,0.0,0.986000,10.614,14.758912,18.150320,34.631995,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,0.0,0.0,2.280150,6.464192,5.142864,0.6140,0.3860,0.0,0.986000,10.614,15.201003,18.625480,35.015253,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,0.0,0.0,2.361478,6.645155,5.327252,0.6359,0.3641,0.0,0.986000,10.614,15.743188,19.077887,35.425005,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,0.0,0.0,1.914030,5.507526,4.285235,0.5153,0.4847,0.0,1.085083,7.776,12.760202,16.481524,26.802744,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,0.0,0.0,2.121379,6.070872,4.752718,0.5710,0.4290,0.0,1.085083,7.776,14.142528,17.889888,27.841597,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,0.0,0.0,2.191997,6.205077,4.927041,0.5895,0.4106,0.0,1.085083,7.776,14.613312,18.225401,28.228980,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,0.0,0.0,2.033869,5.768014,4.505306,0.5458,0.4542,0.0,1.085083,7.776,13.559127,17.132744,27.291791,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [14]:
sorted(list(dff['총점수(100)'].unique()))[-1]

53.90366432

In [165]:
# # 특정 행 추출
# dff[(dff['회사명'] == '심텍홀딩스')]

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,job_s,job_g,특허점수,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
4065,심텍홀딩스,2019,0.0,0.0,2.951630,7.860704,11.891731,0.8054,0.1946,155.0,2.108,0,19.677533,407.151760,31.110513,179.812065,0.000000,0.000000,0.000000,0.000000
4066,심텍홀딩스,2020,0.0,0.0,2.669966,7.004504,10.219081,0.7165,0.2835,155.0,2.108,0,17.799771,405.011260,27.393514,177.001551,-1.877762,-2.140500,-3.716999,-2.810514
4067,심텍홀딩스,2021,0.0,0.0,1.895347,5.431763,7.002879,0.5106,0.4894,155.0,2.108,0,12.635648,401.079407,20.246398,171.437989,-5.164123,-3.931853,-7.147116,-5.563562
4068,심텍홀딩스,2022,0.0,0.0,2.071222,5.899799,7.707496,0.5569,0.4431,155.0,2.108,0,13.808150,402.249496,21.812213,172.786517,1.172502,1.170089,1.565816,1.348528
4069,심텍홀딩스,2023,0.0,0.0,2.253451,6.383559,8.621158,0.6097,0.3903,155.0,2.108,0,15.023008,403.458898,23.842573,174.366168,1.214858,1.209401,2.030360,1.579651
4070,심텍홀딩스,2024,0.0,0.0,1.991331,4.538299,7.906739,0.5189,0.4811,155.0,2.108,0,13.275538,398.845749,22.254975,171.544369,-1.747470,-4.613149,-1.587598,-2.821799


In [49]:
# # 열 순서 변경
# df = df[['회사명', '년도', 'bert_e', '녹색점수', '특허점수', 'E점수(100)', 'E점수(100) 차이', 'bert_s', 'job_s', '리콜점수', 'S점수(100)', 'S점수(100) 차이', 'bert_g', 'job_g', '다트점수', 'G점수(100)', 'G점수(100) 차이', '총점수(100)', '총점수(100) 차이']]
# df

In [15]:
dff

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,0.0,0.0,2.170324,6.191243,4.884456,0.5843,0.4157,0.0,0.986000,10.614,14.468826,17.943107,34.441013,24.846023,NaN,NaN,NaN,NaN
1,3S,2020,0.0,0.0,2.188394,6.250700,4.930433,0.5898,0.4102,0.0,0.986000,10.614,14.589293,18.091751,34.543185,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,0.0,0.0,2.213837,6.274128,4.970398,0.5964,0.4036,0.0,0.986000,10.614,14.758912,18.150320,34.631995,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,0.0,0.0,2.280150,6.464192,5.142864,0.6140,0.3860,0.0,0.986000,10.614,15.201003,18.625480,35.015253,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,0.0,0.0,2.361478,6.645155,5.327252,0.6359,0.3641,0.0,0.986000,10.614,15.743188,19.077887,35.425005,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,0.0,0.0,1.914030,5.507526,4.285235,0.5153,0.4847,0.0,1.085083,7.776,12.760202,16.481524,26.802744,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,0.0,0.0,2.121379,6.070872,4.752718,0.5710,0.4290,0.0,1.085083,7.776,14.142528,17.889888,27.841597,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,0.0,0.0,2.191997,6.205077,4.927041,0.5895,0.4106,0.0,1.085083,7.776,14.613312,18.225401,28.228980,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,0.0,0.0,2.033869,5.768014,4.505306,0.5458,0.4542,0.0,1.085083,7.776,13.559127,17.132744,27.291791,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [16]:
# 결측치 0으로 채우기
dff['E점수(100) 차이'] = dff['E점수(100) 차이'].fillna(0)
dff['S점수(100) 차이'] = dff['S점수(100) 차이'].fillna(0)
dff['G점수(100) 차이'] = dff['G점수(100) 차이'].fillna(0)
dff['총점수(100) 차이'] = dff['총점수(100) 차이'].fillna(0)
dff

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,0.0,0.0,2.170324,6.191243,4.884456,0.5843,0.4157,0.0,0.986000,10.614,14.468826,17.943107,34.441013,24.846023,0.000000,0.000000,0.000000,0.000000
1,3S,2020,0.0,0.0,2.188394,6.250700,4.930433,0.5898,0.4102,0.0,0.986000,10.614,14.589293,18.091751,34.543185,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,0.0,0.0,2.213837,6.274128,4.970398,0.5964,0.4036,0.0,0.986000,10.614,14.758912,18.150320,34.631995,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,0.0,0.0,2.280150,6.464192,5.142864,0.6140,0.3860,0.0,0.986000,10.614,15.201003,18.625480,35.015253,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,0.0,0.0,2.361478,6.645155,5.327252,0.6359,0.3641,0.0,0.986000,10.614,15.743188,19.077887,35.425005,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,0.0,0.0,1.914030,5.507526,4.285235,0.5153,0.4847,0.0,1.085083,7.776,12.760202,16.481524,26.802744,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,0.0,0.0,2.121379,6.070872,4.752718,0.5710,0.4290,0.0,1.085083,7.776,14.142528,17.889888,27.841597,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,0.0,0.0,2.191997,6.205077,4.927041,0.5895,0.4106,0.0,1.085083,7.776,14.613312,18.225401,28.228980,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,0.0,0.0,2.033869,5.768014,4.505306,0.5458,0.4542,0.0,1.085083,7.776,13.559127,17.132744,27.291791,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [17]:
dff.isnull().sum()

회사명            0
년도             0
녹색점수           0
리콜점수           0
bert_e         0
bert_s         0
bert_g         0
Company_Pos    0
Company_Neg    0
특허점수           0
job_s          0
job_g          0
E점수(100)       0
S점수(100)       0
G점수(100)       0
총점수(100)       0
E점수(100) 차이    0
S점수(100) 차이    0
G점수(100) 차이    0
총점수(100) 차이    0
dtype: int64

In [18]:
dff.to_csv('총점수.csv', index=False)

db에 저장

In [20]:
company_detail = list(zip(dff['회사명'], dff['년도'], dff['Company_Pos'], dff['Company_Neg'], dff['E점수(100)'], dff['S점수(100)'], dff['G점수(100)'], dff['총점수(100)'], dff['E점수(100) 차이'], dff['S점수(100) 차이'], dff['G점수(100) 차이'], dff['총점수(100) 차이']))

db.insert_many(
    'tb_company_detail',
    'company, Year, Company_Pos, Company_Neg, E_Score, S_Score, G_Score, Total_Score, E_Score_Change, S_Score_Change, G_Score_Change, Total_Score_Change',
    company_detail
)

True

In [19]:
# # 인덱스 초기화
# db.truncate('tb_company_detail')

True